In [3]:
import cv2
import numpy as np

## Test Cases Paths

In [4]:
test_image_paths = [
    r"Test Cases-20241123/01 - lol easy.jpg",
    r"Test Cases-20241123/02 - still easy.jpg",
    r"Test Cases-20241123/03 - eda ya3am ew3a soba3ak mathazarsh.jpg",
    r"Test Cases-20241123/04 - fen el nadara.jpg",
    r"Test Cases-20241123/05 - meen taffa el nour!!!.jpg",
    r"Test Cases-20241123/06 - meen fata7 el nour 333eenaaayy.jpg",
    r"Test Cases-20241123/07 - mal7 w felfel.jpg",
    r"Test Cases-20241123/08 - compresso espresso.jpg",
    r"Test Cases-20241123/09 - e3del el soora ya3ammm.jpg",
    r"Test Cases-20241123/10 - wen el kontraastttt.jpg",
    r"Test Cases-20241123/11 - bayza 5ales di bsara7a.jpg",
]

## Identify dim images from a list of image file paths and save them in a list.

In [5]:
def find_dim_images(image_paths, threshold=30):

    dim_images = []

    for image_path in image_paths:
        # Read the image in grayscale mode
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Check if the image was loaded successfully
        if image is None:
            print(f"Error: Could not load image {image_path}")
            continue

        # Calculate the average brightness
        avg_brightness = np.mean(image)

        # Check if the image is dim
        if avg_brightness < threshold:
            dim_images.append(image_path)

    return dim_images

# Example
dim_images = find_dim_images(test_image_paths)
print("Dim images:", dim_images)

Dim images: ['Test Cases-20241123/05 - meen taffa el nour!!!.jpg']


## Identify low-contrast images from a list of image file paths, skipping very dim and very bright images.

In [6]:
def find_low_contrast_images(image_paths, dim_threshold=30, bright_threshold=220, contrast_threshold=50):

    low_contrast_images = []

    for image_path in image_paths:
        # Read the image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Check if the image was loaded successfully
        if image is None:
            print(f"Error: Could not load image {image_path}")
            continue

        # Calculate the average brightness
        avg_brightness = np.mean(image)

        # Skip dim images
        if avg_brightness < dim_threshold:
            print(f"Skipping dim image: {image_path}")
            continue

        # Skip very bright images
        if avg_brightness > bright_threshold:
            print(f"Skipping bright image: {image_path}")
            continue

        # Calculate the intensity range (max - min pixel value)
        intensity_range = np.max(image) - np.min(image)

        # Check if the image is low contrast
        if intensity_range < contrast_threshold:
            low_contrast_images.append(image_path)

    return low_contrast_images
low_contrast_images = find_low_contrast_images(test_image_paths)
print("Low-contrast images:", low_contrast_images)

Skipping bright image: Test Cases-20241123/01 - lol easy.jpg
Skipping bright image: Test Cases-20241123/02 - still easy.jpg
Skipping bright image: Test Cases-20241123/03 - eda ya3am ew3a soba3ak mathazarsh.jpg
Skipping bright image: Test Cases-20241123/04 - fen el nadara.jpg
Skipping dim image: Test Cases-20241123/05 - meen taffa el nour!!!.jpg
Skipping bright image: Test Cases-20241123/06 - meen fata7 el nour 333eenaaayy.jpg
Skipping bright image: Test Cases-20241123/09 - e3del el soora ya3ammm.jpg
Low-contrast images: ['Test Cases-20241123/10 - wen el kontraastttt.jpg']


## Flag very bright images where the barcode is not clear due to uniform brightness.

In [12]:
def flag_unclear_bright_images(image_paths, bright_threshold=250, variance_threshold=50):
    flagged_images = []

    for image_path in image_paths:
        # Read the image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Check if the image was loaded successfully
        if image is None:
            print(f"Error: Could not load image {image_path}")
            continue

        # Calculate the average brightness
        avg_brightness = np.mean(image)

        # Calculate the variance in brightness
        brightness_variance = np.var(image)

        # Define a region of interest (ROI) for the barcode (assuming bottom center)
        h, w = image.shape
        roi = image[int(h * 0.8):, int(w * 0.3):int(w * 0.7)]  # Bottom 20%, middle 40%

        # Calculate the average brightness and variance in the ROI
        roi_avg_brightness = np.mean(roi)
        roi_brightness_variance = np.var(roi)

        # Flag image if it's too bright and lacks contrast
        if avg_brightness > bright_threshold and roi_brightness_variance < variance_threshold:
            flagged_images.append(image_path)

    return flagged_images

flagged_images = flag_unclear_bright_images(test_image_paths)
print("Flagged images:", flagged_images)

Flagged images: ['Test Cases-20241123/06 - meen fata7 el nour 333eenaaayy.jpg']


In [15]:
def flag_obscured_barcodes(image_paths, roi_height_ratio=0.2, roi_width_ratio=0.4, disruption_threshold=2):
    flagged_images = []

    for image_path in image_paths:
        # Read the image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Error: Could not load image {image_path}")
            continue

        # Get image dimensions and define ROI
        h, w = image.shape
        roi = image[int(h * (1 - roi_height_ratio)):, int(w * (1 - roi_width_ratio) / 2):int(w * (1 + roi_width_ratio) / 2)]

        # Edge detection to highlight barcode lines
        edges = cv2.Canny(roi, 50, 150)

        # Count vertical lines (sum along columns)
        vertical_line_strength = np.sum(edges, axis=0)

        # Check for disruptions in the line strength
        line_disruption = np.std(vertical_line_strength) / np.mean(vertical_line_strength)
        print(f"Image: {image_path}")
        print(f"  Line Disruption Ratio: {line_disruption:.2f}")

        # Flag if disruption exceeds the threshold
        if line_disruption > disruption_threshold:
            flagged_images.append(image_path)

    return flagged_images

flagged_images = flag_obscured_barcodes(test_image_paths)
print("Flagged images:", flagged_images)


Image: Test Cases-20241123/01 - lol easy.jpg
  Line Disruption Ratio: nan
Image: Test Cases-20241123/02 - still easy.jpg
  Line Disruption Ratio: nan
Image: Test Cases-20241123/03 - eda ya3am ew3a soba3ak mathazarsh.jpg
  Line Disruption Ratio: 2.36
Image: Test Cases-20241123/04 - fen el nadara.jpg
  Line Disruption Ratio: 2.75
Image: Test Cases-20241123/05 - meen taffa el nour!!!.jpg
  Line Disruption Ratio: nan
Image: Test Cases-20241123/06 - meen fata7 el nour 333eenaaayy.jpg
  Line Disruption Ratio: nan
Image: Test Cases-20241123/07 - mal7 w felfel.jpg
  Line Disruption Ratio: 0.12
Image: Test Cases-20241123/08 - compresso espresso.jpg
  Line Disruption Ratio: 1.03
Image: Test Cases-20241123/09 - e3del el soora ya3ammm.jpg
  Line Disruption Ratio: nan
Image: Test Cases-20241123/10 - wen el kontraastttt.jpg
  Line Disruption Ratio: nan
Image: Test Cases-20241123/11 - bayza 5ales di bsara7a.jpg
  Line Disruption Ratio: 0.98
Flagged images: ['Test Cases-20241123/03 - eda ya3am ew3a so

C:\Users\omarb\AppData\Local\Temp\ipykernel_14796\3965685916.py:22: RuntimeWarning: invalid value encountered in scalar divide
  line_disruption = np.std(vertical_line_strength) / np.mean(vertical_line_strength)
